## CNN
- 사용 데이터: CIFAR10
- 데이터 크기: (60000, 32, 32) 

이 튜토리얼은 CIFAR10 숫자를 분류하기 위해 합성곱 신경망(Convolutional Neural Network, CNN)을 훈련합니다. <br>
이 모델을 CIFAR10 테스트 세트에서  71%정확도를 달성합니다.

### 라이브러리 임포트하기
- torch 라이브러리 install을 위해 참고할 사이트: https://pytorch.org/get-started/locally/
- numpy 라이브러리 install을 위해 사용한 코드: `pip install numpy`

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import random

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True

## 데이터 로드
- 데이터 불러오기(pytorch에 내장된 데이터, 처음 실행 시 지정 폴더에 다운됨)
- 데이터 구성을, 학습/테스트 형태로 구분
- MNIST 데이터와 같이 비전 모델 개발에 자주 사용되는 데이터는 학습, 테스트 데이터 구성 이미 정의


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((16, 16)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

batch_size = 128
learning_rate = 0.001

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### CNN 구현
- 모델은 두 개의 합성곱 레이어 (conv1 및 conv2), 맥스 풀링 레이어 (pool), 그리고 두 개의 완전 연결 레이어 (fc1 및 fc2)로 구성됩니다.
- 각 합성곱 레이어 이후에는 ReLU 활성화 함수가 적용됩니다.
- 출력은 분류를 위해 로그 소프트맥스 활성화 함수를 통과합니다.

### 모델 객체 생성
- 모델을 훈련 모드로 설정하고 (model.train()), 배치를 반복하며 손실을 계산하고 역전파를 통해 모델 파라미터를 업데이트합니다.
- 모델을 평가 모드로 설정하고 (model.eval()), 그래디언트를 업데이트하지 않고 손실과 정확도를 계산합니다.

### 모델 학습 메소드 구현
- device: GPU 사용 여부



In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, 3, stride=1, padding=1)  
        self.conv2 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, stride=2, padding=0)
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 43)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 256 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train(model, device, train_loader, optimizer, epoch):   
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):   # 전체 데이터를 barch size만큼 읽어오는 과정 반복
        data, target = data.to(device), target.to(device)       # GPU 사용시 CPU -> GPU
        optimizer.zero_grad()       # Gradient 초기화, 매 학습시마다 초기화 후 backward
        output = model(data)        # 데이터를 MLP에 통과치켜 output 획득
        loss = criterion(output, target)        # output 벡터와 정답을 이용해 손실값 계산
        loss.backward()             # 계산된 손실값으로 backward 진행(변화량 계산, 미분 계산)
        optimizer.step()            # 파라미터 업데이트
        if batch_idx % 100 == 0:    # 결과값 10번 반복마다 print
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}({100.*batch_idx/len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')



def test(model, device, test_loader):
    model.eval()        # 모델 평가 시, eval() 적용해 평가모드 진입(학습시 필요 연산 비활성화)
    test_loss = 0       # 테스트 데이터에 대한 누적 손실값 계산을 위한 변수
    correct = 0         # 테스트 데이터들 중, 정답을 맞춘 데이터의 수를 세기 위한 변수
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()       # 손실함수를 통한 손실값 계산
            pred = output.argmax(dim=1, keepdim=True)           # 0~9중 가장 높은 확률에 해당하는 값
            correct += pred.eq(target.view_as(pred)).sum().item()   # 예측치와 실제 닶이 같은지 비교

    test_loss /= len(test_loader.dataset)           # 전체 데이터 수로 누적값을 나눠, 평균 손실값 계산
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy:{correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n', end='\r')
    # 전체 데이터에 대해 정답을 맞춘 비율(정확도) 출력


# 모델 저장 및 불러오기 함수


In [4]:
def save_model(model, epoch):
    torch.save(model.state_dict(), 'models/mnist_mlp_model{}.pth'.format(epoch))

def load_model(model, model_path):
    model.load_state_dict(torch.load(model_path))

## 학습과 저장
다음 코드 결과 .pth로 저장 / models 파일 생성 해야 함

In [5]:
if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    epochs = 5   

    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)
        save_model(model, epoch)

Train Epoch: 1 [0/50000(0%)]	Loss: 3.769187
Train Epoch: 1 [12800/50000(26%)]	Loss: 1.399992
Train Epoch: 1 [25600/50000(51%)]	Loss: 1.357658
Train Epoch: 1 [38400/50000(77%)]	Loss: 1.274638

Test set: Average loss: 0.0090, Accuracy:5959/10000 (60%)
Train Epoch: 2 [0/50000(0%)]	Loss: 1.050650
Train Epoch: 2 [12800/50000(26%)]	Loss: 1.009012
Train Epoch: 2 [25600/50000(51%)]	Loss: 1.018012
Train Epoch: 2 [38400/50000(77%)]	Loss: 1.032301

Test set: Average loss: 0.0078, Accuracy:6504/10000 (65%)
Train Epoch: 3 [0/50000(0%)]	Loss: 0.883776
Train Epoch: 3 [12800/50000(26%)]	Loss: 1.004939
Train Epoch: 3 [25600/50000(51%)]	Loss: 0.757790
Train Epoch: 3 [38400/50000(77%)]	Loss: 0.927450

Test set: Average loss: 0.0072, Accuracy:6798/10000 (68%)
Train Epoch: 4 [0/50000(0%)]	Loss: 0.894832
Train Epoch: 4 [12800/50000(26%)]	Loss: 0.936428
Train Epoch: 4 [25600/50000(51%)]	Loss: 0.743931
Train Epoch: 4 [38400/50000(77%)]	Loss: 0.776116

Test set: Average loss: 0.0069, Accuracy:6958/10000 (70%)


# 개선 방안 및 시도
### 1. 파라미터 및 함수 변경
- 최적화 함수: Adam(model.parameters(), lr=0.001)
- 손실함수: CrossEntropyLoss()
- batch_size: 128

> - 함수 및 파라미터 변경 전 정확도: 5282/10000 (53%)
> - 함수 및 파라미터 변경 후 정화도: 7098/10000 (71%)
>> 결과: 기존 변경 전과 비교하여 약 2000개의 데이터를 잘 맞췄고 약 18%의 정확도가 증가하는 유의미한 결과를 확인할 수 있었다. <br> 이 결과를 통해 파라미터, 함수가 모델 생성 과정과 결과적으로 미치는 영향에 대해 체감하게 되었다. 